# Socioeconomic Data Analysis With SQL  

Loading the ipython-sql  extension and establishing a connection with the Db2 database

In [13]:
import csv, sqlite3

con = sqlite3.connect("RealWorldData.db")
cur = con.cursor()

In [10]:
!pip install -q pandas==1.1.5

In [ ]:
!pip install ipython-sql

In [16]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [18]:
%sql sqlite:///RealWorldData.db

The dataset that will be analyzed is available as a .CSV file. To analyse it using SQL two steps are followed. Reading the csv files from the given url into pandas dataframes and then using the df.to_sql() function to convert each csv file to a table in SQlite with the CSV data loaded in it. 

In [19]:
import pandas
df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv")
df.to_sql("CENSUS_DATA", con, if_exists='replace', index=False,method="multi")

df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv")
df.to_sql("CHICAGO_CRIME_DATA", con, if_exists='replace', index=False, method="multi")

df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv")
df.to_sql("CHICAGO_PUBLIC_SCHOOLS_DATA", con, if_exists='replace', index=False, method="multi")

/Users/mazin/anaconda3/lib/python3.11/site-packages/pandas/core/generic.py:2605: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


Loading tables in Schema to check data retrieval and table creation was successful 

In [20]:
%sql SELECT name FROM sqlite_master WHERE type='table'

 * sqlite:///RealWorldData.db
Done.


name
CENSUS_DATA
CHICAGO_CRIME_DATA
CHICAGO_PUBLIC_SCHOOLS_DATA


### 1. Finding the total number of crimes recorded in the Crime table 

In [21]:
%sql SELECT count(*) FROM CHICAGO_CRIME_DATA;

 * sqlite:///RealWorldData.db
Done.


count(*)
533


### 2. Listing the community areas with per capita income less than 11000

In [27]:
%sql SELECT COMMUNITY_AREA_NAME, PER_CAPITA_INCOME FROM CENSUS_DATA WHERE PER_CAPITA_INCOME < 11000;

 * sqlite:///RealWorldData.db
Done.


COMMUNITY_AREA_NAME,PER_CAPITA_INCOME
West Garfield Park,10934
South Lawndale,10402
Fuller Park,10432
Riverdale,8201


### 3. Listing all case numbers for crimes involving minors

In [37]:
%sql SELECT CASE_NUMBER, DESCRIPTION FROM CHICAGO_CRIME_DATA WHERE DESCRIPTION LIKE '%minor%';

 * sqlite:///RealWorldData.db
Done.


CASE_NUMBER,DESCRIPTION
HL266884,SELL/GIVE/DEL LIQUOR TO MINOR
HK238408,ILLEGAL CONSUMPTION BY MINOR


### 4. Listing all kidnapping crimes involving a child

In [44]:
%sql SELECT CASE_NUMBER, PRIMARY_TYPE, DESCRIPTION FROM CHICAGO_CRIME_DATA WHERE DESCRIPTION LIKE '%Child%' AND PRIMARY_TYPE LIKE '%kidnapping%';

 * sqlite:///RealWorldData.db
Done.


CASE_NUMBER,PRIMARY_TYPE,DESCRIPTION
HN144152,KIDNAPPING,CHILD ABDUCTION/STRANGER


### 5. Listing types of crimes at schools

In [48]:
%sql SELECT PRIMARY_TYPE, LOCATION_DESCRIPTION FROM CHICAGO_CRIME_DATA WHERE LOCATION_DESCRIPTION LIKE '%school%';

 * sqlite:///RealWorldData.db
Done.


PRIMARY_TYPE,LOCATION_DESCRIPTION
BATTERY,"SCHOOL, PUBLIC, GROUNDS"
BATTERY,"SCHOOL, PUBLIC, BUILDING"
BATTERY,"SCHOOL, PUBLIC, BUILDING"
BATTERY,"SCHOOL, PUBLIC, BUILDING"
BATTERY,"SCHOOL, PUBLIC, GROUNDS"
CRIMINAL DAMAGE,"SCHOOL, PUBLIC, GROUNDS"
NARCOTICS,"SCHOOL, PUBLIC, GROUNDS"
NARCOTICS,"SCHOOL, PUBLIC, BUILDING"
ASSAULT,"SCHOOL, PUBLIC, GROUNDS"
CRIMINAL TRESPASS,"SCHOOL, PUBLIC, GROUNDS"


### 6. Calculating the average saftey score for all types of schools

In [50]:
%%sql

SELECT AVG(SAFETY_SCORE) AS Average_Safety_Score
FROM CHICAGO_PUBLIC_SCHOOLS_DATA;

 * sqlite:///RealWorldData.db
Done.


Average_Safety_Score
49.50487329434698


### 7. Listing the top 5 community areas with the highest percentage of Households below the poverty line

In [57]:
%%sql 

SELECT COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY FROM CENSUS_DATA 
ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC 
LIMIT 5;

 * sqlite:///RealWorldData.db
Done.


COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


### 8. Listing the community area that is most crime prone 

In [69]:
%%sql 

SELECT COMMUNITY_AREA_NUMBER, COUNT(*) AS Crime_Count
FROM CHICAGO_CRIME_DATA 
GROUP BY COMMUNITY_AREA_NUMBER
ORDER BY Crime_Count DESC
LIMIT 1;





 * sqlite:///RealWorldData.db
Done.


COMMUNITY_AREA_NUMBER,Crime_Count
25.0,43


### 9. Using a subquery to list the community area with the highest hardship index 

In [68]:
%%sql

SELECT COMMUNITY_AREA_NAME, HARDSHIP_INDEX 
FROM CENSUS_DATA
WHERE HARDSHIP_INDEX = (SELECT MAX(HARDSHIP_INDEX) FROM CENSUS_DATA);




 * sqlite:///RealWorldData.db
Done.


COMMUNITY_AREA_NAME,HARDSHIP_INDEX
Riverdale,98.0


### 10. Using a subquery to determine the community area name with the most number of crimes

In [71]:
%%sql 

SELECT COMMUNITY_AREA_NAME
FROM CENSUS_DATA
WHERE COMMUNITY_AREA_NUMBER = (
    SELECT COMMUNITY_AREA_NUMBER
    FROM CHICAGO_CRIME_DATA
    GROUP BY COMMUNITY_AREA_NUMBER
    ORDER BY COUNT(*) DESC
    LIMIT 1
);

 * sqlite:///RealWorldData.db
Done.


COMMUNITY_AREA_NAME
Austin
